# 최근 영업일 기준 데이터 받기

In [1]:
import requests as rq 
from bs4 import BeautifulSoup

url = 'https://finance.naver.com/sise/sise_deposit.nhn'
data = rq.get(url)
data_html = BeautifulSoup(data.content)
parse_day = data_html.select_one('div.subplot_sise_graph2 > ul.subtop_chart_note > li > span.tah').text

AttributeError: 'NoneType' object has no attribute 'text'